#### If come back to this notebook or have to restart the notebook, start from Check Point. The preprocessed data has been saved to disk.

In [ ]:
%matplotlib inline
import cv2
import os
import random
import numpy as np
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

## Preparing the Data

In [ ]:
TRAIN_DIR = 'input/train/'
train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]

In [ ]:
# Explore the dataset
image_id = 10000

# Don't modify below
image_example = cv2.imread(train_images[image_id])
print("Stats of Dog_vs_Cat Training Dataset")
print("Total Images: {}".format(len(train_images)))
print("Dog:          {}".format(len([i for i in train_images if 'dog' in i ])))
print("Cat:          {}".format(len([i for i in train_images if 'cat' in i ])))
print("")
print("Example of Image {}".format(image_id))
print("Image - Shape: {}".format(image_example.shape))
print("Image - Min Value: {}, Max Value: {}".format(image_example.min(), image_example.max()))
print("Label - {}".format('dog' if 'dog' in train_images[image_id] else 'cat'))
display(Image.open(train_images[image_id]))

In [ ]:
#Randomize Data
random.shuffle(train_images)

In [ ]:
# Resize images
ROWS = 64 #Resized image row size
COLS = 64 #Resized image columns size
CHANNELS = 3 #RGB channels

# Don't modify below
feature = []
for i, img in enumerate(train_images): 
    img = cv2.imread(img)
    img = cv2.resize(img, dsize=(ROWS, COLS))
    feature.append(img)
feature = np.array(feature, dtype=np.uint8)

In [ ]:
# Explore the resized image
image_example_new = 10

#Don't modify below
print('Resized Image')
display(Image.fromarray(feature[image_example_new]))
print('')
print('Original Image')
display(Image.open(train_images[image_example_new]))

In [ ]:
# Normalize features
feature = (feature-np.min(feature))/(np.max(feature)-np.min(feature))

In [ ]:
#One-hot encoding on the label
label = [[0, 1] if 'cat' in i else [1, 0] for i in train_images]
label = np.array(label, dtype=np.uint8)

In [ ]:
import pickle
with open('preprocess_image.p', 'wb') as f:
    pickle.dump((feature,label), f)

## Check Point

If come back to this notebook or have to restart the notebook, start from here. The preprocessed data has been saved to disk.

In [1]:
import pickle
import numpy as np
import tensorflow as tf

In [2]:
with open('preprocess_image.p', 'rb') as f:
    feature, label = pickle.load(f)

In [3]:
print("Image - Shape: {}".format(feature.shape[1:]))

Image - Shape: (64, 64, 3)


In [4]:
# Train test split to generate training set and validation set
from sklearn.model_selection import train_test_split
training_size = 0.9 # 90% of the data will be used as training, 
                    #and the remaining 10% will be used as validation set
train_feature, val_feature, train_label, val_label=train_test_split(feature, label, train_size=training_size, random_state=42)

C:\Users\Zaoli\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
def conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides):
    """
    Apply convolution then max pooling to x_tensor
    :param x_tensor: TensorFlow Tensor
    :param conv_num_outputs: Number of outputs for the convolutional layer
    :param conv_ksize: kernal size 2-D Tuple for the convolutional layer
    :param conv_strides: Stride 2-D Tuple for convolution
    :param pool_ksize: kernal size 2-D Tuple for pool
    :param pool_strides: Stride 2-D Tuple for pool
    : return: A tensor that represents convolution and max pooling of x_tensor
    """
    weight =  tf.Variable(tf.random_normal(conv_ksize+(x_tensor.shape[-1].value,)+(conv_num_outputs,),
                                           stddev=0.05))
    bias = tf.Variable(tf.zeros([conv_num_outputs]))
    x = tf.nn.conv2d(x_tensor, weight, strides=(1,)+conv_strides+(1,), padding='SAME')
    x = tf.nn.bias_add(x, bias)
    x = tf.nn.relu(x)
    x = tf.nn.max_pool(x, ksize=((1,)+ pool_ksize + (1,)),
                      strides=((1,)+ pool_strides + (1,)),
                      padding='SAME')
    return x 

In [6]:
def flatten(x_tensor):
    """
    Flatten x_tensor to (Batch Size, Flattened Image Size)
    : x_tensor: A tensor of size (Batch Size, ...), where ... are the image dimensions.
    : return: A tensor of size (Batch Size, Flattened Image Size).
    """
    flatten_x = tf.reshape(x_tensor, 
                           shape=[tf.shape(x_tensor)[0], x_tensor.shape[1].value*x_tensor.shape[2].value*x_tensor.shape[3].value])
    return flatten_x

In [7]:
def fully_conn(x_tensor, num_outputs):
    """
    Apply a fully connected layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    weight = tf.Variable(tf.truncated_normal((x_tensor.shape[-1].value, num_outputs),
                                            stddev = 0.05))
    bias = tf.Variable(tf.zeros([num_outputs]))
    fc_x = tf.add(tf.matmul(x_tensor, weight), bias)
    fc_x = tf.nn.relu(fc_x)
    
    return fc_x

In [8]:
def output(x_tensor, num_outputs):
    """
    Apply a output layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    # TODO: Implement Function
    weight = tf.Variable(tf.truncated_normal([x_tensor.shape[-1].value,num_outputs],
                                            stddev=0.05))
    bias = tf.Variable(tf.zeros([num_outputs]))
    return tf.add(tf.matmul(x_tensor, weight), bias)

In [9]:
def conv_net(x, keep_prob):
    """
    Create a convolutional neural network model
    : x: Placeholder tensor that holds image data.
    : keep_prob: Placeholder tensor that hold dropout keep probability.
    : return: Tensor that represents logits
    """
    # Function Definition from Above:
    #    conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides)
    conv = conv2d_maxpool(x, 32, (4,4), (1,1), (3,3), (2,2))

    # Function Definition from Above:
    #   flatten(x_tensor)
    flt = flatten(conv)

    # Function Definition from Above:
    #   fully_conn(x_tensor, num_outputs)
    fc = fully_conn(flt, 512)
    fc = tf.nn.dropout(fc, keep_prob)

    # Function Definition from Above:
    #   output(x_tensor, num_outputs)
    
    # TODO: return output
    return output(fc, 2)

In [10]:
##############################
## Build the Neural Network ##
##############################

# Remove previous weights, bias, inputs, etc..
import tensorflow as tf

tf.reset_default_graph()

# Inputs
ROWS = feature.shape[1]
COLS = feature.shape[2] #Resized image columns size
CHANNELS = feature.shape[3] #RGB channels
image_shape = (ROWS, COLS, CHANNELS)
n_classes = label.shape[1]
x = tf.placeholder(tf.float32, shape = (None, image_shape[0], image_shape[1], image_shape[2]), name='x')
y = tf.placeholder(tf.float32, shape = (None, n_classes), name='y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

# Model
logits = conv_net(x, keep_prob)

# Name logits Tensor, so that is can be loaded from disk after training
logits = tf.identity(logits, name='logits')

# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y), name='cost')
optimizer = tf.train.AdamOptimizer().minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

In [11]:
# Tune Parameters
epochs = 12
batch_size = 256
keep_probability = 0.3

In [14]:
save_model_path = './dog_vs_cat'

print('Training...')
with tf.Session() as sess:
    #Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    # Training cycle
    for epoch in range(epochs):
        # Loop over all batches
        for batch_start in range(0, train_feature.shape[0], batch_size):
            batch_end = min(batch_start + batch_size, feature.shape[0])
            feature_batch = train_feature[batch_start: batch_end]
            label_batch = train_label[batch_start: batch_end]

            sess.run(optimizer, feed_dict={
                    x: feature_batch,
                    y: label_batch,
                    keep_prob: keep_probability
                })
            print('Epoch {:>2}, Batch {}: '.format(epoch + 1, int(batch_start/batch_size + 1)))
            loss = sess.run(cost, feed_dict={
                    x: feature_batch,
                    y: label_batch,
                    keep_prob: 1.
                })
            
            valid_loss = sess.run(cost, feed_dict={
                    x: val_feature,
                    y: val_label,
                    keep_prob: 1.
                })
            
            valid_acc = sess.run(accuracy, feed_dict={
                    x: val_feature,
                    y: val_label,
                    keep_prob: 1.
                })
            print('Training Loss: {:>3.4f}, Validation Loss: {:>3.4f}, Validation Accuracy: {:.2f}%'.format(loss, valid_loss, valid_acc*100))
    
    # Save model
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)

Training...
Epoch  1, Batch 1: 
Training Loss: 13.3315, Validation Loss: 14.6200, Validation Accuracy: 50.44%
Epoch  1, Batch 2: 
Training Loss: 4.5915, Validation Loss: 4.3737, Validation Accuracy: 50.44%
Epoch  1, Batch 3: 
Training Loss: 2.2623, Validation Loss: 2.2462, Validation Accuracy: 49.56%
Epoch  1, Batch 4: 
Training Loss: 2.5073, Validation Loss: 2.5780, Validation Accuracy: 49.56%
Epoch  1, Batch 5: 
Training Loss: 1.9971, Validation Loss: 2.0622, Validation Accuracy: 49.56%
Epoch  1, Batch 6: 
Training Loss: 1.3078, Validation Loss: 1.3892, Validation Accuracy: 49.56%
Epoch  1, Batch 7: 
Training Loss: 0.8735, Validation Loss: 0.8919, Validation Accuracy: 49.56%
Epoch  1, Batch 8: 
Training Loss: 0.7030, Validation Loss: 0.7096, Validation Accuracy: 43.24%
Epoch  1, Batch 9: 
Training Loss: 0.7395, Validation Loss: 0.7476, Validation Accuracy: 50.32%
Epoch  1, Batch 10: 
Training Loss: 0.7631, Validation Loss: 0.8006, Validation Accuracy: 50.44%
Epoch  1, Batch 11: 
Trai

Training Loss: 0.6681, Validation Loss: 0.6763, Validation Accuracy: 60.36%
Epoch  1, Batch 87: 
Training Loss: 0.6709, Validation Loss: 0.6757, Validation Accuracy: 61.36%
Epoch  1, Batch 88: 
Training Loss: 0.6668, Validation Loss: 0.6750, Validation Accuracy: 62.08%
Epoch  2, Batch 1: 
Training Loss: 0.6725, Validation Loss: 0.6743, Validation Accuracy: 62.08%
Epoch  2, Batch 2: 
Training Loss: 0.6591, Validation Loss: 0.6738, Validation Accuracy: 61.68%
Epoch  2, Batch 3: 
Training Loss: 0.6638, Validation Loss: 0.6728, Validation Accuracy: 61.84%
Epoch  2, Batch 4: 
Training Loss: 0.6759, Validation Loss: 0.6716, Validation Accuracy: 61.72%
Epoch  2, Batch 5: 
Training Loss: 0.6698, Validation Loss: 0.6705, Validation Accuracy: 62.08%
Epoch  2, Batch 6: 
Training Loss: 0.6726, Validation Loss: 0.6699, Validation Accuracy: 62.44%
Epoch  2, Batch 7: 
Training Loss: 0.6585, Validation Loss: 0.6688, Validation Accuracy: 61.88%
Epoch  2, Batch 8: 
Training Loss: 0.6639, Validation Loss

Training Loss: 0.6140, Validation Loss: 0.6145, Validation Accuracy: 66.32%
Epoch  2, Batch 84: 
Training Loss: 0.5904, Validation Loss: 0.6100, Validation Accuracy: 67.44%
Epoch  2, Batch 85: 
Training Loss: 0.5750, Validation Loss: 0.6130, Validation Accuracy: 67.52%
Epoch  2, Batch 86: 
Training Loss: 0.5858, Validation Loss: 0.6132, Validation Accuracy: 67.52%
Epoch  2, Batch 87: 
Training Loss: 0.5790, Validation Loss: 0.6117, Validation Accuracy: 67.32%
Epoch  2, Batch 88: 
Training Loss: 0.5956, Validation Loss: 0.6098, Validation Accuracy: 67.28%
Epoch  3, Batch 1: 
Training Loss: 0.6133, Validation Loss: 0.6083, Validation Accuracy: 67.72%
Epoch  3, Batch 2: 
Training Loss: 0.5562, Validation Loss: 0.6085, Validation Accuracy: 67.56%
Epoch  3, Batch 3: 
Training Loss: 0.5856, Validation Loss: 0.6096, Validation Accuracy: 67.32%
Epoch  3, Batch 4: 
Training Loss: 0.6161, Validation Loss: 0.6070, Validation Accuracy: 67.88%
Epoch  3, Batch 5: 
Training Loss: 0.5760, Validation L

Training Loss: 0.4862, Validation Loss: 0.5671, Validation Accuracy: 70.32%
Epoch  3, Batch 81: 
Training Loss: 0.4856, Validation Loss: 0.5665, Validation Accuracy: 70.60%
Epoch  3, Batch 82: 
Training Loss: 0.5137, Validation Loss: 0.5643, Validation Accuracy: 70.36%
Epoch  3, Batch 83: 
Training Loss: 0.5686, Validation Loss: 0.5699, Validation Accuracy: 70.20%
Epoch  3, Batch 84: 
Training Loss: 0.5461, Validation Loss: 0.5645, Validation Accuracy: 70.40%
Epoch  3, Batch 85: 
Training Loss: 0.5127, Validation Loss: 0.5635, Validation Accuracy: 70.44%
Epoch  3, Batch 86: 
Training Loss: 0.5029, Validation Loss: 0.5623, Validation Accuracy: 70.80%
Epoch  3, Batch 87: 
Training Loss: 0.5297, Validation Loss: 0.5619, Validation Accuracy: 70.76%
Epoch  3, Batch 88: 
Training Loss: 0.5299, Validation Loss: 0.5612, Validation Accuracy: 70.56%
Epoch  4, Batch 1: 
Training Loss: 0.5586, Validation Loss: 0.5607, Validation Accuracy: 70.64%
Epoch  4, Batch 2: 
Training Loss: 0.4970, Validatio

Training Loss: 0.4754, Validation Loss: 0.5330, Validation Accuracy: 72.56%
Epoch  4, Batch 78: 
Training Loss: 0.4699, Validation Loss: 0.5321, Validation Accuracy: 72.76%
Epoch  4, Batch 79: 
Training Loss: 0.4840, Validation Loss: 0.5344, Validation Accuracy: 72.24%
Epoch  4, Batch 80: 
Training Loss: 0.4387, Validation Loss: 0.5423, Validation Accuracy: 72.56%
Epoch  4, Batch 81: 
Training Loss: 0.4425, Validation Loss: 0.5387, Validation Accuracy: 72.24%
Epoch  4, Batch 82: 
Training Loss: 0.4715, Validation Loss: 0.5313, Validation Accuracy: 72.52%
Epoch  4, Batch 83: 
Training Loss: 0.5241, Validation Loss: 0.5408, Validation Accuracy: 72.00%
Epoch  4, Batch 84: 
Training Loss: 0.5223, Validation Loss: 0.5381, Validation Accuracy: 72.48%
Epoch  4, Batch 85: 
Training Loss: 0.4837, Validation Loss: 0.5324, Validation Accuracy: 72.68%
Epoch  4, Batch 86: 
Training Loss: 0.4575, Validation Loss: 0.5348, Validation Accuracy: 72.24%
Epoch  4, Batch 87: 
Training Loss: 0.4933, Validat

...

Training Loss: 0.3907, Validation Loss: 0.5072, Validation Accuracy: 75.00%
Epoch  8, Batch 66: 
Training Loss: 0.3885, Validation Loss: 0.5203, Validation Accuracy: 74.44%
Epoch  8, Batch 67: 
Training Loss: 0.4464, Validation Loss: 0.5116, Validation Accuracy: 74.68%
Epoch  8, Batch 68: 
Training Loss: 0.3694, Validation Loss: 0.4972, Validation Accuracy: 75.16%
Epoch  8, Batch 69: 
Training Loss: 0.3922, Validation Loss: 0.5036, Validation Accuracy: 74.84%
Epoch  8, Batch 70: 
Training Loss: 0.4448, Validation Loss: 0.5099, Validation Accuracy: 74.68%
Epoch  8, Batch 71: 
Training Loss: 0.4189, Validation Loss: 0.5141, Validation Accuracy: 74.68%
Epoch  8, Batch 72: 
Training Loss: 0.4124, Validation Loss: 0.5044, Validation Accuracy: 75.12%
Epoch  8, Batch 73: 
Training Loss: 0.4392, Validation Loss: 0.4944, Validation Accuracy: 74.92%
Epoch  8, Batch 74: 
Training Loss: 0.3519, Validation Loss: 0.4950, Validation Accuracy: 75.24%
Epoch  8, Batch 75: 
Training Loss: 0.4326, Validat

Training Loss: 0.4172, Validation Loss: 0.4992, Validation Accuracy: 75.56%
Epoch  9, Batch 63: 
Training Loss: 0.3411, Validation Loss: 0.4987, Validation Accuracy: 75.60%
Epoch  9, Batch 64: 
Training Loss: 0.3472, Validation Loss: 0.4984, Validation Accuracy: 75.52%
Epoch  9, Batch 65: 
Training Loss: 0.3709, Validation Loss: 0.5022, Validation Accuracy: 75.24%
Epoch  9, Batch 66: 
Training Loss: 0.3692, Validation Loss: 0.5131, Validation Accuracy: 74.84%
Epoch  9, Batch 67: 
Training Loss: 0.4122, Validation Loss: 0.5066, Validation Accuracy: 75.64%
Epoch  9, Batch 68: 
Training Loss: 0.3516, Validation Loss: 0.4946, Validation Accuracy: 75.52%
Epoch  9, Batch 69: 
Training Loss: 0.3720, Validation Loss: 0.4927, Validation Accuracy: 75.60%
Epoch  9, Batch 70: 
Training Loss: 0.3966, Validation Loss: 0.4922, Validation Accuracy: 75.76%
Epoch  9, Batch 71: 
Training Loss: 0.3883, Validation Loss: 0.4954, Validation Accuracy: 75.72%
Epoch  9, Batch 72: 
Training Loss: 0.3795, Validat

Training Loss: 0.3428, Validation Loss: 0.4992, Validation Accuracy: 76.44%
Epoch 10, Batch 60: 
Training Loss: 0.3650, Validation Loss: 0.5052, Validation Accuracy: 76.12%
Epoch 10, Batch 61: 
Training Loss: 0.3558, Validation Loss: 0.5005, Validation Accuracy: 76.36%
Epoch 10, Batch 62: 
Training Loss: 0.3793, Validation Loss: 0.5005, Validation Accuracy: 76.12%
Epoch 10, Batch 63: 
Training Loss: 0.3280, Validation Loss: 0.4978, Validation Accuracy: 76.32%
Epoch 10, Batch 64: 
Training Loss: 0.3354, Validation Loss: 0.4950, Validation Accuracy: 75.48%
Epoch 10, Batch 65: 
Training Loss: 0.3629, Validation Loss: 0.4963, Validation Accuracy: 75.56%
Epoch 10, Batch 66: 
Training Loss: 0.3556, Validation Loss: 0.5024, Validation Accuracy: 75.64%
Epoch 10, Batch 67: 
Training Loss: 0.4059, Validation Loss: 0.5037, Validation Accuracy: 75.36%
Epoch 10, Batch 68: 
Training Loss: 0.3332, Validation Loss: 0.5019, Validation Accuracy: 75.52%
Epoch 10, Batch 69: 
Training Loss: 0.3587, Validat

Training Loss: 0.3447, Validation Loss: 0.5068, Validation Accuracy: 75.48%
Epoch 11, Batch 57: 
Training Loss: 0.3627, Validation Loss: 0.5187, Validation Accuracy: 75.20%
Epoch 11, Batch 58: 
Training Loss: 0.3081, Validation Loss: 0.5241, Validation Accuracy: 74.56%
Epoch 11, Batch 59: 
Training Loss: 0.3284, Validation Loss: 0.5108, Validation Accuracy: 75.68%
Epoch 11, Batch 60: 
Training Loss: 0.3359, Validation Loss: 0.5096, Validation Accuracy: 76.12%
Epoch 11, Batch 61: 
Training Loss: 0.3561, Validation Loss: 0.5168, Validation Accuracy: 75.48%
Epoch 11, Batch 62: 
Training Loss: 0.3671, Validation Loss: 0.5196, Validation Accuracy: 75.08%
Epoch 11, Batch 63: 
Training Loss: 0.3207, Validation Loss: 0.5036, Validation Accuracy: 75.56%
Epoch 11, Batch 64: 
Training Loss: 0.3164, Validation Loss: 0.4897, Validation Accuracy: 76.28%
Epoch 11, Batch 65: 
Training Loss: 0.3532, Validation Loss: 0.4868, Validation Accuracy: 76.36%
Epoch 11, Batch 66: 
Training Loss: 0.3520, Validat

Training Loss: 0.2822, Validation Loss: 0.5096, Validation Accuracy: 76.36%
Epoch 12, Batch 54: 
Training Loss: 0.2844, Validation Loss: 0.5108, Validation Accuracy: 76.16%
Epoch 12, Batch 55: 
Training Loss: 0.3710, Validation Loss: 0.5037, Validation Accuracy: 76.40%
Epoch 12, Batch 56: 
Training Loss: 0.3048, Validation Loss: 0.4955, Validation Accuracy: 76.64%
Epoch 12, Batch 57: 
Training Loss: 0.3136, Validation Loss: 0.5010, Validation Accuracy: 76.12%
Epoch 12, Batch 58: 
Training Loss: 0.2818, Validation Loss: 0.5109, Validation Accuracy: 75.88%
Epoch 12, Batch 59: 
Training Loss: 0.3043, Validation Loss: 0.4976, Validation Accuracy: 76.48%
Epoch 12, Batch 60: 
Training Loss: 0.3208, Validation Loss: 0.4973, Validation Accuracy: 76.80%
Epoch 12, Batch 61: 
Training Loss: 0.3457, Validation Loss: 0.5286, Validation Accuracy: 75.68%
Epoch 12, Batch 62: 
Training Loss: 0.3619, Validation Loss: 0.5558, Validation Accuracy: 74.32%
Epoch 12, Batch 63: 
Training Loss: 0.3025, Validat